In [56]:
import requests
import numpy as np
import pandas as pd
import cx_Oracle
import os

# Oracle database connection details
username = os.getenv('Oracle_Username')
password = os.getenv('Oracle_Password')
hostname = '10.0.0.69'
port = '1521'
service_name = 'XEPDB1'

# Construct the connection string
dsn = cx_Oracle.makedsn(hostname, port, service_name=service_name)
connection = cx_Oracle.connect(username, password, dsn)

# Define the SQL query
query = '''
SELECT DISTINCT franchise_id FROM dim_team
'''

# Create a cursor object
cursor = connection.cursor()
cursor.execute(query)

# Fetch the results
results = cursor.fetchall()

# Create a Pandas Series to store the distinct franchise_id values
franchise_ids = pd.Series(dtype=object)
for row in results:
    franchise_id = row[0]
    franchise_ids = franchise_ids.append(pd.Series(franchise_id))

#DROP ANY BLANK VALUES
franchise_ids = franchise_ids.dropna()

#SORT VALUES FROM ACSENDING ORDER
franchise_ids = franchise_ids.sort_values()

# URL for franchises
url = "https://statsapi.web.nhl.com/api/v1/franchises/"

# Create an empty DataFrame to store the results
result_df = pd.DataFrame()

# Loop through the URLs
for url_suffix in franchise_ids.astype(str):
    full_url = url + url_suffix
    
    # Send the request
    r = requests.get(full_url)
    
    # Check if the request was successful
    if r.status_code == 200:
        json_data = r.json()
        
        # Extract the data from the JSON response
        data = json_data.get('franchises')
        
        # Normalize the JSON data into a DataFrame
        df = pd.json_normalize(data=data, sep="_")
        
        # Append the DataFrame to the result DataFrame
        result_df = result_df.append(df, ignore_index=True)
    else:
        print(f"Request for URL {full_url} failed with status code {r.status_code}")
        
col_dict = {"franchiseId":"franchise_id",
            "firstSeasonId":"first_season_id",
            "mostRecentTeamId":"most_recent_team_id",
            "teamName":"team_name",
            "locationName":"location_name",
            "link":"franchise_link",
            "lastSeasonId":"last_season_id"}

# Rename columns
result_df = result_df.rename(columns=col_dict)
result_df.columns = result_df.columns.str.upper()

#FILL ALL NA VALUES WITH EMPTY STRING
result_df = result_df.fillna('')

#CONVERT ALL COLUMNS TO STRING
result_df = result_df.astype(str)

# Define the target table name
table_name = 'dim_franchise_load'

# Drop the table if it already exists
drop_query = f"DROP TABLE {table_name}"
try:
    cursor.execute(drop_query)
    print(f"Dropped table {table_name}")
except cx_Oracle.DatabaseError as e:
    error_code, error_message = e.args[0].code, e.args[0].message
    if error_code != 942:  # ORA-00942: table or view does not exist
        print(f"Error occurred while dropping table {table_name}: {error_message}")

# Read the SQL file
sql_file = r"C:\Users\Jeremy\Documents\nhl_analysis\02_Source_Code\table_create_scripts\dim_franchise_load.sql"
with open(sql_file, 'r') as file:
    create_table_query = file.read()

# Create the table
cursor.execute(create_table_query)
print(f"Created table {table_name}")

# Convert dataframe to a list of tuples
data_tuples = [tuple(row) for row in result_df.values]

# Prepare the INSERT query with placeholders for the columns
insert_query = f"INSERT INTO {table_name} " \
               f"({', '.join(result_df.columns)}) " \
               f"VALUES ({', '.join([':' + str(i+1) for i in range(len(result_df.columns))])})"

# Insert data into the target table
cursor.executemany(insert_query, data_tuples)
print(f"Inserted {len(data_tuples)} rows into {table_name}")

# Commit the changes
connection.commit()

# Read the SQL script
update_file = r"C:\Users\Jeremy\Documents\nhl_analysis\02_Source_Code\update_scripts\dim_franchise_update.sql"
with open(update_file, 'r') as file:
    script = file.read()

# Split the script into individual statements
statements = script.split(';')

# Execute each statement
for statement in statements:
    if statement.strip():  # Skip empty statements
        try:
            cursor.execute(statement)
            rows_affected = cursor.rowcount
            print(f"Rows affected: {rows_affected}")
        except cx_Oracle.DatabaseError as e:
            error, = e.args
            error_message = error.message
            print(f"Error executing statement: {error_message}")

# Commit the changes
connection.commit()

# Drop the table if it already exists
drop_query = f"DROP TABLE {table_name}"
try:
    cursor.execute(drop_query)
    print(f"Dropped table {table_name}")
except cx_Oracle.DatabaseError as e:
    error_code, error_message = e.args[0].code, e.args[0].message
    if error_code != 942:  # ORA-00942: table or view does not exist
        print(f"Error occurred while dropping table {table_name}: {error_message}")

# Close the cursor and the database connection
cursor.close()
connection.close()

<ipython-input-56-6b5f0e99130b>:34: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  franchise_ids = franchise_ids.append(pd.Series(franchise_id))


Dropped table dim_franchise_load
Created table dim_franchise_load
Inserted 39 rows into dim_franchise_load
Rows affected: 39
Rows affected: 39
Rows affected: 39
Dropped table dim_franchise_load
